In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('/content/movies.csv')

In [5]:
df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [6]:
import re

In [7]:
def clean_title(title):
  title=re.sub("[^a-zA-Z0-9 ]", "", title)
  return title

In [8]:
df['clean_title']=df['title'].apply(clean_title)

In [9]:
df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  \
0  Adventure|Animation|Children|Comedy|Fantasy   
1                   Adventure|Children|Fantasy   
2                               Comedy|Romance   
3                         Comedy|Drama|Romance   
4                                       Comedy   

                        clean_title  
0                    Toy Story 1995  
1                      Jumanji 1995  
2             Grumpier Old Men 1995  
3            Waiting to Exhale 1995  
4  Father of the Bride Part II 1995

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(1,2))
tdfi=vectorizer.fit_transform(df['clean_title'])

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
def search(title):
  tiltle=clean_title(title)
  que_vect=vectorizer.transform([tiltle])
  similarity=cosine_similarity(que_vect,tdfi).flatten()
  indices=np.argpartition(similarity,-5)[-5:]
  result=df.iloc[indices][::-1]

  return result


In [13]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy story', description='Movie Title:')

Output()

In [ ]:
ratings=pd.read_csv('/content/ratings.csv')

In [ ]:
ratings

userId  movieId  rating   timestamp
0             1      296     5.0  1147880044
1             1      306     3.5  1147868817
2             1      307     5.0  1147868828
3             1      665     5.0  1147878820
4             1      899     3.5  1147868510
...         ...      ...     ...         ...
3854855   25388     3260     4.0  1171217819
3854856   25388     3266     4.5  1170972070
3854857   25388     3267     3.0  1171217816
3854858   25388     3274     4.0  1171217811
3854859   25388     3317     4.0  1171217802

[3854860 rows x 4 columns]

In [ ]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [ ]:
movie_id=1

In [ ]:
similar_users=ratings[(ratings['movieId']==movie_id) & (ratings['rating'] > 4)]['userId'].unique()

In [ ]:
similar_users

array([   36,    75,    86, ..., 25354, 25361, 25381])

In [ ]:
similar_user_recs=ratings[(ratings['userId'].isin(similar_users)) & (ratings['rating'] > 4)]['movieId']

In [ ]:
similar_user_recs

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
3854238    179401
3854240    182823
3854241    183611
3854243    187593
3854244    192003
Name: movieId, Length: 211865, dtype: int64

In [ ]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)


In [ ]:
similar_user_recs = similar_user_recs[similar_user_recs > .1]

In [ ]:
similar_user_recs

movieId
1       1.000000
318     0.432157
260     0.383311
296     0.351764
356     0.351764
          ...   
1208    0.101764
5418    0.101425
745     0.101085
953     0.100746
1307    0.100746
Name: count, Length: 106, dtype: float64

In [ ]:
all_users = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings['rating'] > 4)]

In [ ]:
all_users

userId  movieId  rating   timestamp
0             1      296     5.0  1147880044
29            1     4973     4.5  1147869080
48            1     7361     5.0  1147880055
72            2      110     5.0  1141416589
76            2      260     5.0  1141417172
...         ...      ...     ...         ...
3854801   25388     2329     4.5  1170972131
3854812   25388     2571     4.5  1170971820
3854826   25388     2762     5.0  1170971551
3854836   25388     2959     4.5  1170971440
3854844   25388     3147     4.5  1171217503

[257661 rows x 4 columns]

In [ ]:
all_users_recs=all_users['movieId'].value_counts() / len(all_users['userId'].unique())

In [ ]:
all_users_recs

movieId
318      0.340703
296      0.284563
2571     0.245226
356      0.232284
593      0.227319
           ...   
953      0.044004
50872    0.039124
745      0.035645
78499    0.034584
2355     0.024739
Name: count, Length: 106, dtype: float64

In [ ]:
rec_percentage=pd.concat([similar_user_recs , all_users_recs] ,axis=1)
rec_percentage.columns=['similar','all']

In [ ]:
rec_percentage

similar       all
movieId                    
1        1.000000  0.125095
318      0.432157  0.340703
260      0.383311  0.218068
296      0.351764  0.284563
356      0.351764  0.232284
...           ...       ...
1208     0.101764  0.079861
5418     0.101425  0.062293
745      0.101085  0.035645
953      0.100746  0.044004
1307     0.100746  0.046338

[106 rows x 2 columns]

In [ ]:
rec_percentage['score']= rec_percentage['similar'] / rec_percentage['all']
rec_percentage=rec_percentage.sort_values('score',ascending=False)

In [ ]:
rec_percentage

similar       all     score
movieId                              
1        1.000000  0.125095  7.993894
3114     0.281886  0.053976  5.222426
2355     0.111601  0.024739  4.511134
78499    0.144166  0.034584  4.168595
588      0.222185  0.067937  3.270456
...           ...       ...       ...
296      0.351764  0.284563  1.236157
2329     0.114993  0.093737  1.226768
79132    0.160448  0.131079  1.224057
4973     0.131275  0.110413  1.188946
2959     0.250339  0.214546  1.166830

[106 rows x 3 columns]

In [ ]:
rec_percentage.head(10).merge(df,left_index=True,right_on='movieId')

similar       all     score  movieId  \
0      1.000000  0.125095  7.993894        1   
3021   0.281886  0.053976  5.222426     3114   
2264   0.111601  0.024739  4.511134     2355   
14813  0.144166  0.034584  4.168595    78499   
580    0.222185  0.067937  3.270456      588   
4780   0.219810  0.070313  3.126158     4886   
6258   0.211669  0.069422  3.049016     6377   
587    0.187924  0.061826  3.039545      595   
8246   0.193691  0.068022  2.847482     8961   
729    0.101085  0.035645  2.835929      745   

                                        title  \
0                            Toy Story (1995)   
3021                       Toy Story 2 (1999)   
2264                     Bug's Life, A (1998)   
14813                      Toy Story 3 (2010)   
580                            Aladdin (1992)   
4780                    Monsters, Inc. (2001)   
6258                      Finding Nemo (2003)   
587               Beauty and the Beast (1991)   
8246                  Incredibles, The (2004)   
729    Wallace & Gromit: A Close Shave (1995)   

                                                 genres  \
0           Adventure|Animation|Children|Comedy|Fantasy   
3021        Adventure|Animation|Children|Comedy|Fantasy   
2264                Adventure|Animation|Children|Comedy   
14813  Adventure|Animation|Children|Comedy|Fantasy|IMAX   
580         Adventure|Animation|Children|Comedy|Musical   
4780        Adventure|Animation|Children|Comedy|Fantasy   
6258                Adventure|Animation|Children|Comedy   
587     Animation|Children|Fantasy|Musical|Romance|IMAX   
8246         Action|Adventure|Animation|Children|Comedy   
729                           Animation|Children|Comedy   

                              clean_title  
0                          Toy Story 1995  
3021                     Toy Story 2 1999  
2264                     Bugs Life A 1998  
14813                    Toy Story 3 2010  
580                          Aladdin 1992  
4780                    Monsters Inc 2001  
6258                    Finding Nemo 2003  
587             Beauty and the Beast 1991  
8246                 Incredibles The 2004  
729    Wallace  Gromit A Close Shave 1995

In [ ]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(df, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [ ]:
movie_name_input = widgets.Text(
    value='Toy story',
    description='Movie Title:',
    disabled=False
)
recomendation_list =widgets.Output()
def on_type(data):
  with recomendation_list:
    recomendation_list.clear_output()
    title=data['new']
    if len(title) > 5:
      result=search(title)
      movie_id=result.iloc[0]['movieId']
      display(find_similar_movies(movie_id))
movie_name_input.observe(on_type,names='value')
display(movie_name_input,recomendation_list)

Text(value='Toy story', description='Movie Title:')

Output()

In [ ]:
import pickle

In [ ]:
with open('movie_recommender.pkl','wb') as f:
  pickle.dump(find_similar_movies,f)

In [ ]:
with open('movie_recommender.pkl','rb') as f:
  recommendation=pickle.load(f)

In [ ]:
recommendation(100)

score                       title                        genres
98   586.189189            City Hall (1996)                Drama|Thriller
630  127.432432           Diabolique (1996)                Drama|Thriller
78    63.128067           Juror, The (1996)                Drama|Thriller
60    55.827542       Eye for an Eye (1996)                Drama|Thriller
92    32.931977      Beautiful Girls (1996)          Comedy|Drama|Romance
636   26.444625   Courage Under Fire (1996)        Action|Crime|Drama|War
489   22.670300   Executive Decision (1996)     Action|Adventure|Thriller
277   20.353791  Murder in the First (1995)                Drama|Thriller
618   17.990461          Primal Fear (1996)  Crime|Drama|Mystery|Thriller
789   17.718565      Time to Kill, A (1996)                Drama|Thriller